In [7]:
from bitarray import bitarray

#Show the progression of a convolutional shift register and the parity bits generated in each state

#Jamie Taylor KK6OLF
#January 2018

#Released under CCO - have fun with it.
#WSPR encoding in Python by Jamie Taylor
#To the extent possible under law, the person who associated CC0 with
#WSPR encoding in Python has waived all copyright and related or neighboring rights to WSPR encoding in Python.
#You should have received a copy of the CC0 legalcode along with this work.  
#If not, see <http://creativecommons.org/publicdomain/zero/1.0/>.

K = 3 #constraint length
k = 1 #input bits...never used...assumed to be 1 in this demo
r = 2 #parity bits

#the taps in the convolution filter (i.e., the taps on the shift reg)
polynomials = [bitarray("111"),bitarray("101")] 

#sanity check the parameters
assert(len(polynomials) == r) #r must equal to the number of polynomials for parity bit generation

for poly in polynomials:
    assert(len(poly) == K) #the polynomials must be the same as the constraint length

In [8]:
def paritybits(polylist, shiftreg):
    parity = []
    
    for poly in polylist:
        tmpreg = shiftreg & poly #for each tap, see if there is a bit set in the shift register
        p = int(tmpreg.count(1) % 2) #modulo2 addition of the tap values (number of set bits mod 2)
        parity.append(p)
        
    return parity #return as many parity bits as there are polynomials describing the shiftreg taps

In [9]:
def encode(message, polylist):
    
    constraint_length = polylist[0].length()
    #create a shift register K in size, initialize to all zeros
    sr = bitarray(str("{:0"+str(constraint_length)+"b}").format(0))
    
    output = []
    
    #push each message bit into the shift register and generate parity bits
    for i in range(0, len(message)):
        
        #move the shift register one bit to the right
        for s in range(len(sr)-1, 0, -1):
            sr[s] = sr[s-1]
            
        #shift the next message bit into the shift register
        sr[0] = message[i]
        
        #capture intermediate values for demonstration purposes
        print ""
        print "shift reg state: " + str(sr.to01())
        print "partity bits for this state: " + str(paritybits(polylist, sr))
        
        #generate the convolutional parity bits for this bit
        output.extend(paritybits(polylist, sr))
        
    return output

#this message shows the convolutional filters "impulse response to '1'"
inputmessage = bitarray("100")

tx_message = encode(inputmessage, polynomials)

print ""
print "final encoded message:"
print tx_message


shift reg state: 100
partity bits for this state: [1, 1]

shift reg state: 010
partity bits for this state: [1, 0]

shift reg state: 001
partity bits for this state: [1, 1]

final encoded message:
[1, 1, 1, 0, 1, 1]
